<a href="https://colab.research.google.com/github/dolmani38/similarly/blob/master/BERT_Sentence_similarity_0720.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
!nvidia-smi

Mon Jul 20 13:46:45 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.51.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P0    72W / 149W |   2987MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [45]:
import pandas as pd
# 한국어 영화 리뷰를 Load.
ds = pd.read_csv("https://raw.githubusercontent.com/dolmani38/similarly/master/ratings_test.txt",sep='\t')
ds

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0
...,...,...,...
49995,4608761,오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함,1
49996,5308387,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따 OOOO,0
49997,9072549,그림도 좋고 완성도도 높았지만... 보는 내내 불안하게 만든다,0
49998,5802125,절대 봐서는 안 될 영화.. 재미도 없고 기분만 잡치고.. 한 세트장에서 다 해먹네,0


##Sentence Transformers: Multilingual Sentence Embeddings using BERT / RoBERTa / XLM-RoBERTa & Co. with PyTorch

참조:https://github.com/UKPLab/sentence-transformers

In [46]:
!pip install -U sentence-transformers

Requirement already up-to-date: sentence-transformers in /usr/local/lib/python3.6/dist-packages (0.3.0)


##영어 + 한국어 BERT 모델의 사용

#Extending Sentence Embeddings Models to New Languages

#Available Pre-trained Models


참조:https://github.com/UKPLab/sentence-transformers/blob/master/docs/pretrained-models/multilingual-models.md

In [47]:
from sentence_transformers import SentenceTransformer
# embedder download...
embedder = SentenceTransformer('xlm-r-large-en-ko-nli-ststb')

In [48]:
#영화 리뷰의 내용을 corpus array에 저장한다.

corpus = []
for row in ds.iterrows():
    doc_id = row[1][0]
    doc_cont = str(row[1][1])
    corpus.append(doc_cont)

print('corpus size:',len(corpus))

corpus size: 50000


In [49]:
#하나의 sentence를 embedding해 본다.
#약 123ms 소요되며, 1024 차원의 vector를 생성한다.

from timeit import default_timer as timer
from datetime import timedelta
start = timer()

sentence_embedding = embedder.encode([corpus[4]],show_progress_bar=True)   

end = timer()

print("End... elapsed time : {}".format(timedelta(seconds=end-start)))
print('vector dim:',len(sentence_embedding[0]))
sentence_embedding


End... elapsed time : 0:00:00.113676
vector dim: 1024


[array([ 0.6906828 , -0.2728697 ,  0.541603  , ..., -0.58096015,
         0.6411979 , -0.16210863], dtype=float32)]

In [50]:
corpus[4]

'3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??'

In [56]:
# 50000개 전체 corpus를 embedding한다.

start = timer()
corpus_embeddings = embedder.encode(corpus[0:1000],show_progress_bar=True) 
end = timer()

print("End... elapsed time : {}".format(timedelta(seconds=end-start)))


End... elapsed time : 0:00:26.901627


In [57]:
# 유사한 sentence를 찾는 함수의 정의
import scipy

def searchSentence(queries,Threshold_score=0.60, Topn=0):
    index = {}
    n = len(queries)
    query_embeddings = embedder.encode(queries)
    for query, query_embedding in zip(queries, query_embeddings):
        distances = scipy.spatial.distance.cdist([query_embedding], corpus_embeddings, "cosine")[0]

        results = zip(range(len(distances)), distances)
        results = sorted(results, key=lambda x: x[1])
        
        for idx, distance in results[0:len(results) if Topn==0 else Topn]:
          score = 1-distance
          if not idx in index:
            index[idx] = 0.0
          index[idx] += float(score/n)

    return index

In [58]:
# Quary를 넣고 유사한 리뷰를 찾아 낸다.

query = ['존나 재밌어']
rank_count = 20
start = timer()
idx = searchSentence(query, Topn=rank_count)
end = timer()
print("End... elapsed time : {}".format(timedelta(seconds=end-start)))
ds.loc[list(idx.keys()),'score'] = list(idx.values())
ds.loc[list(idx.keys())]

End... elapsed time : 0:00:00.048353


,id,document,label,score
396,7792908,재밌어요.,1,0.963795
30,1418173,너무너무 재밌다,1,0.949169
913,3544246,넘 재밌다.ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ,1,0.937082
906,10143343,재밋엇어요오오오오몽·으,1,0.923444
359,7616705,재밌었음,1,0.918170
825,68780,잼있음,1,0.913131
991,7666739,난 재밌던데 ㅋㅋ,1,0.908700
294,7165384,진짜 재밌게 봤다,1,0.901853
300,376428,재미있는데여,1,0.897588
374,9390521,재미있어요 너무나도재밌어요,1,0.896785


## Local sensitive hash를 이용한 유사 문자 탐색

참조:https://en.wikipedia.org/wiki/Locality-sensitive_hashing

참조:https://github.com/pixelogik/NearPy

In [59]:
# 이번에는 Local sensitive hash를 이용한다.
!pip install NearPy

     |████████████████████████████████| 71kB 4.1MB/s 
     |████████████████████████████████| 61kB 8.0MB/s 
  Created wheel for bitarray: filename=bitarray-1.4.2-cp36-cp36m-linux_x86_64.whl size=108530 sha256=50544ebd87749d77459b0d752dac9bae847b2981226e0a9e3fcdf86baa972211
  Stored in directory: /root/.cache/pip/wheels/21/11/f3/7c9571443475b1a364ae541cf3cc80f9ceb4f6d6dc5ba7db44
Successfully built bitarray


In [163]:
from nearpy import Engine
from nearpy.hashes import RandomBinaryProjections
from nearpy.distances import CosineDistance
import numpy as np

# Dimension of our vector space
dimension = len(corpus_embeddings[0])

# Create a random binary hash
rbp = RandomBinaryProjections('rbp', 10)

# Create engine with pipeline configuration
engine = Engine(dimension, lshashes=[rbp],distance=CosineDistance())

In [164]:
for index in range(len(corpus_embeddings)):
    engine.store_vector(corpus_embeddings[index],index)

In [166]:

query = ['존나 재밌어']
query_embeddings = embedder.encode(query)
# Get nearest neighbours
start = timer()
N = engine.neighbours(query_embeddings[0])
end = timer()
print("End... elapsed time : {}".format(timedelta(seconds=end-start)))
pN = np.asarray(N)
pN.shape

ds.loc[list(pN[:,1]),'score'] = list(pN[:,2])
ds.loc[list(pN[:,1])]



End... elapsed time : 0:00:00.000665


,id,document,label,score
396,7792908,재밌어요.,1,0.036205
30,1418173,너무너무 재밌다,1,0.050831
906,10143343,재밋엇어요오오오오몽·으,1,0.076556
359,7616705,재밌었음,1,0.081830
825,68780,잼있음,1,0.086869
991,7666739,난 재밌던데 ㅋㅋ,1,0.091300
300,376428,재미있는데여,1,0.102412
599,6983606,재미있어요~~,1,0.114951
976,7443281,정말재밌게봤었는데......,1,0.122679
312,23174,재미있어요,1,0.154376
